In [1]:
import sys
from pathlib import Path
# get current path
sys.path.append(str(Path.cwd().parent))

from indra_gpt.benchmarks.benchmark import Benchmark
from indra_gpt.resources.constants import INPUT_DEFAULT


INFO: [2025-02-12 17:46:14] indra.preassembler.grounding_mapper.disambiguate - INDRA DB is not available for text content retrieval for grounding disambiguation.
/Users/thomaslim/miniconda3/envs/indra_gpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = 'gpt-4o-mini'
benchmark_file = INPUT_DEFAULT
structured_output = False
n_statements = 5
benchmark = Benchmark(model, benchmark_file, structured_output, n_statements)


In [3]:
benchmark_df = benchmark.get_benchmark_df()


Extracting:   0%|          | 0/5 [00:00<?, ?statement/s]INFO: [2025-02-12 17:46:21] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-12 17:46:26] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-12 17:46:31] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Extracting:  20%|██        | 1/5 [00:14<00:58, 14.62s/statement]INFO: [2025-02-12 17:46:36] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-12 17:48:11] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-12 17:48:23] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Extracting:  40%|████      | 2/5 [02:06<03:35, 71.72s/statement]INFO: [2025-02-12 17:48:30] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO

5
5
5
5


In [4]:
benchmark_df


,original_statements_json,generated_statements_json,original_statements,generated_statements,results
0,"{'type': 'Acetylation', 'enz': {'name': 'C5', ...","{""type"": ""Activation"", ""subj"": {""name"": ""C5a"",...","[Acetylation(C5(), STAT3())]","Activation(C5a(), STAT3(), proliferation)",Error: 'list' object has no attribute 'equals'
1,"{'type': 'Acetylation', 'enz': {'name': 'KANSL...","{""type"": ""Activation"", ""subj"": {""name"": ""MOF a...","[Acetylation(KANSL1(), TP53(), K, 120)]","Error: Expecting ',' delimiter: line 1 column ...",Error: 'list' object has no attribute 'equals'
2,"{'type': 'Acetylation', 'enz': {'name': 'EP300...","{\n ""type"": ""Acetylation"",\n ""enz"": {\n ""...","[Acetylation(EP300(), TP73())]","Acetylation(p300(), p73())",Error: 'list' object has no attribute 'equals'
3,"{'type': 'Activation', 'subj': {'name': 'BMP',...","{""type"": ""IncreaseAmount"", ""subj"": {""name"": ""B...","[Activation(BMP(), PTEN())]","IncreaseAmount(BMP(), PTEN())",Error: 'list' object has no attribute 'equals'
4,"{'type': 'Activation', 'subj': {'name': 'Caspa...","{""type"": ""Ubiquitination"", ""enz"": {""name"": ""Ca...","[Activation(Caspase(), PRKCZ())]","Ubiquitination(Caspase(), PRKCZ())",Error: 'list' object has no attribute 'equals'
